In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fine-tuning-jsonl-data/fine_tuning_data_train.jsonl
/kaggle/input/fine-tuning-jsonl-data/fine_tuning_data_val.jsonl


In [29]:
#!pip install -q transformers datasets accelerate sentencepiece sacrebleu unbabel-comet evaluate

In [30]:
#pip install --upgrade pyarrow==12.0.1

In [31]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import json
import torch
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Optional
from tqdm.auto import tqdm

# Set memory optimization environment variables
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    get_linear_schedule_with_warmup,
    set_seed
)
from datasets import Dataset, DatasetDict
from accelerate import Accelerator, DistributedDataParallelKwargs
from torch.utils.data import DataLoader
import evaluate

# Set random seed for reproducibility
set_seed(42)

# Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

PyTorch version: 2.8.0+cu126
CUDA available: True
Number of GPUs: 2


In [33]:
@dataclass
class Config:
    """Training configuration"""
    # Model
    model_name: str = "facebook/nllb-200-distilled-600M"
    
    # Languages (NLLB codes)
    src_lang: str = "fra_Latn"  # French
    tgt_lang: str = "arb_Arab"  # Arabic
    
    # Data
    train_file: str = "/kaggle/input/fine-tuning-jsonl-data/fine_tuning_data_train.jsonl" 
    val_file: str = "/kaggle/input/fine-tuning-jsonl-data/fine_tuning_data_val.jsonl"  
    max_length: int = 256
    
    # Training
    num_epochs: int = 5
    per_device_train_batch_size: int = 2 
    per_device_eval_batch_size: int = 4   
    gradient_accumulation_steps: int = 16  
    learning_rate: float = 2e-5
    weight_decay: float = 0.01
    warmup_ratio: float = 0.1
    max_grad_norm: float = 1.0
    
    # Optimization
    fp16: bool = True
    
    # Checkpointing
    output_dir: str = "/kaggle/working/nllb-fr-ar-legal"
    save_steps: int = 5
    eval_steps: int = 5
    logging_steps: int = 2
    
    # Evaluation
    num_beams: int = 3  # Reduced from 5 to save memory
    early_stopping: bool = True
    
    # Memory optimization
    gradient_checkpointing: bool = True

config = Config()
os.makedirs(config.output_dir, exist_ok=True)


In [34]:
def load_jsonl_dataset(file_path: str) -> List[Dict[str, str]]:
    """Load JSONL dataset"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def create_datasets(config: Config):
    """Create train and validation datasets"""
    
    # Load raw data
    print("Loading datasets...")
    train_data = load_jsonl_dataset(config.train_file)
    val_data = load_jsonl_dataset(config.val_file)
    
    print(f"Train samples: {len(train_data)}")
    print(f"Validation samples: {len(val_data)}")
    
    # Create HuggingFace datasets
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    
    return DatasetDict({
        "train": train_dataset,
        "validation": val_dataset
    })

# Load datasets
try:
    raw_datasets = create_datasets(config)
    print("\nExample training sample:")
    print(f"Source: {raw_datasets['train'][0]['src_text'][:100]}...")
    print(f"Target: {raw_datasets['train'][0]['tgt_text'][:100]}...")
except FileNotFoundError:
    print("⚠️  Dataset files not found.")


Loading datasets...
Train samples: 300
Validation samples: 31

Example training sample:
Source: Il n’y a pas d’infraction, ni de peine ou de mesures de sûreté sans loi....
Target: لا جريمة ولا عقوبة أو تدابير أمن بغير قانون.

...


In [35]:
print("Loading tokenizer and model...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    config.model_name,
    src_lang=config.src_lang,
    tgt_lang=config.tgt_lang
)

# Get forced_bos_token_id for Arabic
# NLLB tokenizer stores language codes in different ways
if hasattr(tokenizer, 'lang_code_to_id'):
    forced_bos_token_id = tokenizer.lang_code_to_id[config.tgt_lang]
else:
    # For NllbTokenizerFast, convert language code to token id
    forced_bos_token_id = tokenizer.convert_tokens_to_ids(config.tgt_lang)

print(f"Forced BOS token ID for {config.tgt_lang}: {forced_bos_token_id}")

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(config.model_name)
model.config.forced_bos_token_id = forced_bos_token_id

# Enable gradient checkpointing to save memory
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()
    print("✓ Gradient checkpointing enabled")

print(f"Model parameters: {model.num_parameters():,}")
print(f"Model loaded on CPU (will be moved to GPU by Accelerator)")

Loading tokenizer and model...
Forced BOS token ID for arb_Arab: 256011
✓ Gradient checkpointing enabled
Model parameters: 615,073,792
Model loaded on CPU (will be moved to GPU by Accelerator)


In [36]:
def preprocess_function(examples):
    """Tokenize inputs and targets"""
    # Set source language
    tokenizer.src_lang = config.src_lang
    
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["src_text"],
        max_length=config.max_length,
        truncation=True,
        padding=False  # We'll pad dynamically
    )
    
    # Tokenize targets
    tokenizer.src_lang = config.tgt_lang  # Switch to target for proper encoding
    labels = tokenizer(
        examples["tgt_text"],
        max_length=config.max_length,
        truncation=True,
        padding=False
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization
print("Tokenizing datasets...")
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    desc="Tokenizing"
)

print(f"Tokenized train samples: {len(tokenized_datasets['train'])}")
print(f"Tokenized val samples: {len(tokenized_datasets['validation'])}")

Tokenizing datasets...


Tokenizing:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/31 [00:00<?, ? examples/s]

Tokenized train samples: 300
Tokenized val samples: 31


In [37]:
@dataclass
class DataCollatorForSeq2Seq:
    """Custom data collator with dynamic padding"""
    tokenizer: AutoTokenizer
    model: Optional[AutoModelForSeq2SeqLM] = None
    padding: bool = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    label_pad_token_id: int = -100
    
    def __call__(self, features: List[Dict[str, List[int]]]) -> Dict[str, torch.Tensor]:
        # Separate labels from inputs
        labels = [f["labels"] for f in features] if "labels" in features[0] else None
        
        # Remove labels from features for input padding
        features_without_labels = [{k: v for k, v in f.items() if k != "labels"} for f in features]
        
        # Pad inputs
        batch = self.tokenizer.pad(
            features_without_labels,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt"
        )
        
        # Pad labels
        if labels is not None:
            max_label_length = max(len(l) for l in labels)
            if self.pad_to_multiple_of is not None:
                max_label_length = (
                    (max_label_length + self.pad_to_multiple_of - 1)
                    // self.pad_to_multiple_of
                    * self.pad_to_multiple_of
                )
            
            padded_labels = []
            for label in labels:
                remainder = [self.label_pad_token_id] * (max_label_length - len(label))
                padded_labels.append(label + remainder)
            
            batch["labels"] = torch.tensor(padded_labels)
        
        return batch

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    pad_to_multiple_of=8 if config.fp16 else None
)

In [38]:
# Configure DDP
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=False)

# Initialize accelerator
accelerator = Accelerator(
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    mixed_precision="fp16" if config.fp16 else "no",
    log_with=None,
    kwargs_handlers=[ddp_kwargs]
)

print(f"Distributed type: {accelerator.distributed_type}")
print(f"Number of processes: {accelerator.num_processes}")
print(f"Process index: {accelerator.process_index}")
print(f"Device: {accelerator.device}")

# Create dataloaders
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    batch_size=config.per_device_train_batch_size,
    shuffle=True,
    collate_fn=data_collator,
    num_workers=0,  # Reduced to save memory
    pin_memory=True
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=config.per_device_eval_batch_size,
    shuffle=False,
    collate_fn=data_collator,
    num_workers=0,  # Reduced to save memory
    pin_memory=True
)

Distributed type: DistributedType.NO
Number of processes: 1
Process index: 0
Device: cuda


In [39]:
# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay
)

# Calculate total training steps
num_update_steps_per_epoch = len(train_dataloader) // config.gradient_accumulation_steps
max_train_steps = config.num_epochs * num_update_steps_per_epoch
num_warmup_steps = int(max_train_steps * config.warmup_ratio)

# Scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=max_train_steps
)

print(f"Total training steps: {max_train_steps}")
print(f"Warmup steps: {num_warmup_steps}")
print(f"Steps per epoch: {num_update_steps_per_epoch}")


Total training steps: 45
Warmup steps: 4
Steps per epoch: 9


In [40]:
# Prepare everything with accelerator
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)

print("✓ Model and dataloaders prepared for distributed training")
print(f"✓ Model now on device: {accelerator.device}")


✓ Model and dataloaders prepared for distributed training
✓ Model now on device: cuda


In [41]:
pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [42]:
# Load metrics
bleu_metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    """Post-process predictions and labels for metric computation"""
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]  # BLEU expects list of references
    return preds, labels

def compute_metrics(predictions, references):
    """Compute BLEU score"""
    decoded_preds, decoded_labels = postprocess_text(predictions, references)
    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [43]:
def evaluate_model(model, eval_dataloader, accelerator, config, forced_bos_token_id):
    """Evaluate model on validation set"""
    model.eval()
    
    all_predictions = []
    all_references = []
    total_loss = 0
    
    # Clear cache before evaluation
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Disable gradient checkpointing during evaluation
    if hasattr(model.module if hasattr(model, 'module') else model, 'gradient_checkpointing_disable'):
        (model.module if hasattr(model, 'module') else model).gradient_checkpointing_disable()
    
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating", disable=not accelerator.is_local_main_process):
            # Compute loss (skip if OOM risk is high)
            try:
                outputs = model(**batch)
                loss = outputs.loss
                total_loss += accelerator.gather(loss).mean().item()
            except RuntimeError as e:
                if "out of memory" in str(e):
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    accelerator.print("⚠️  Skipping loss computation due to memory")
                else:
                    raise e
            
            # Generate predictions with reduced beam search
            # Don't pass forced_bos_token_id if it causes device issues
            # The model config already has it set
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=config.max_length,
                num_beams=config.num_beams,
                early_stopping=config.early_stopping,
                use_cache=True  # Enable KV cache for memory efficiency
            )
            
            # Gather predictions and labels from all processes
            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=-100
            )
            
            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()
            
            # Replace -100 in labels (used for padding)
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            
            # Decode
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            all_predictions.extend(decoded_preds)
            all_references.extend(decoded_labels)
    
    # Compute metrics only on main process
    metrics = {}
    if accelerator.is_main_process:
        avg_loss = total_loss / len(eval_dataloader) if total_loss > 0 else 0
        metrics = compute_metrics(all_predictions, all_references)
        metrics["eval_loss"] = avg_loss
    
    # Re-enable gradient checkpointing for training
    if config.gradient_checkpointing:
        if hasattr(model.module if hasattr(model, 'module') else model, 'gradient_checkpointing_enable'):
            (model.module if hasattr(model, 'module') else model).gradient_checkpointing_enable()
    
    model.train()
    return metrics

In [44]:
def train(model, train_dataloader, eval_dataloader, optimizer, lr_scheduler, accelerator, config, forced_bos_token_id):
    """Main training loop"""
    
    global_step = 0
    best_bleu = 0
    
    accelerator.print("=" * 80)
    accelerator.print("Starting training...")
    accelerator.print("=" * 80)
    
    model.train()
    
    for epoch in range(config.num_epochs):
        accelerator.print(f"\nEpoch {epoch + 1}/{config.num_epochs}")
        
        # Clear cache at start of each epoch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        progress_bar = tqdm(
            total=len(train_dataloader),
            disable=not accelerator.is_local_main_process,
            desc=f"Epoch {epoch + 1}"
        )
        
        for step, batch in enumerate(train_dataloader):
            with accelerator.accumulate(model):
                try:
                    outputs = model(**batch)
                    loss = outputs.loss
                    
                    accelerator.backward(loss)
                    
                    if accelerator.sync_gradients:
                        accelerator.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                    
                    optimizer.step()
                    lr_scheduler.step()
                    optimizer.zero_grad()
                    
                except RuntimeError as e:
                    if "out of memory" in str(e):
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()
                        accelerator.print(f"⚠️  OOM at step {global_step}, skipping batch")
                        optimizer.zero_grad()
                        continue
                    else:
                        raise e
            
            if accelerator.sync_gradients:
                global_step += 1
                progress_bar.update(1)
                
                # Logging
                if global_step % config.logging_steps == 0:
                    avg_loss = accelerator.gather(loss).mean().item()
                    accelerator.print(
                        f"Step {global_step} | Loss: {avg_loss:.4f} | LR: {lr_scheduler.get_last_lr()[0]:.2e}"
                    )
                
                # Evaluation
                if global_step % config.eval_steps == 0:
                    accelerator.print("\n" + "=" * 80)
                    accelerator.print(f"Evaluating at step {global_step}...")
                    
                    metrics = evaluate_model(model, eval_dataloader, accelerator, config, forced_bos_token_id)
                    
                    if accelerator.is_main_process:
                        accelerator.print(f"Eval Loss: {metrics['eval_loss']:.4f}")
                        accelerator.print(f"BLEU Score: {metrics['bleu']:.2f}")
                        
                        # Save best model
                        if metrics['bleu'] > best_bleu:
                            best_bleu = metrics['bleu']
                            accelerator.print(f"✓ New best BLEU: {best_bleu:.2f}")
                            
                            # Save checkpoint
                            unwrapped_model = accelerator.unwrap_model(model)
                            unwrapped_model.save_pretrained(
                                config.output_dir,
                                is_main_process=accelerator.is_main_process,
                                save_function=accelerator.save
                            )
                            tokenizer.save_pretrained(config.output_dir)
                            accelerator.print(f"✓ Model saved to {config.output_dir}")
                    
                    accelerator.print("=" * 80 + "\n")
                
                # Save periodic checkpoint
                if global_step % config.save_steps == 0:
                    checkpoint_dir = os.path.join(config.output_dir, f"checkpoint-{global_step}")
                    os.makedirs(checkpoint_dir, exist_ok=True)
                    
                    unwrapped_model = accelerator.unwrap_model(model)
                    unwrapped_model.save_pretrained(
                        checkpoint_dir,
                        is_main_process=accelerator.is_main_process,
                        save_function=accelerator.save
                    )
                    if accelerator.is_main_process:
                        tokenizer.save_pretrained(checkpoint_dir)
                        accelerator.print(f"✓ Checkpoint saved at step {global_step}")
        
        progress_bar.close()
    
    accelerator.print("\n" + "=" * 80)
    accelerator.print("Training completed!")
    accelerator.print(f"Best BLEU score: {best_bleu:.2f}")
    accelerator.print("=" * 80)
    
    return best_bleu

In [45]:
best_bleu = train(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    accelerator=accelerator,
    config=config,
    forced_bos_token_id=forced_bos_token_id
)

Starting training...

Epoch 1/5


Epoch 1:   0%|          | 0/150 [00:00<?, ?it/s]

You're using a NllbTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at ste

Epoch 2:   0%|          | 0/150 [00:00<?, ?it/s]

...

⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at ste

Epoch 4:   0%|          | 0/150 [00:00<?, ?it/s]

⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at ste

Epoch 5:   0%|          | 0/150 [00:00<?, ?it/s]

⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at step 0, skipping batch
⚠️  OOM at ste

In [47]:
if accelerator.is_main_process:
    print("\n" + "=" * 80)
    print("Running final evaluation with COMET...")
    print("=" * 80)
    
    # Load COMET metric
    try:
        comet_metric = evaluate.load("comet")
        
        # Prepare data for COMET
        eval_samples = raw_datasets["validation"].select(range(min(100, len(raw_datasets["validation"]))))
        sources = eval_samples["src_text"]
        references = eval_samples["tgt_text"]
        
        # Get forced_bos_token_id for inference
        if hasattr(tokenizer, 'lang_code_to_id'):
            inference_forced_bos = tokenizer.lang_code_to_id[config.tgt_lang]
        else:
            inference_forced_bos = tokenizer.convert_tokens_to_ids(config.tgt_lang)
        
        # Generate translations
        model.eval()
        predictions = []
        
        for i in tqdm(range(0, len(sources), config.per_device_eval_batch_size), desc="Generating"):
            batch_sources = sources[i:i + config.per_device_eval_batch_size]
            
            tokenizer.src_lang = config.src_lang
            inputs = tokenizer(
                batch_sources,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=config.max_length
            ).to(accelerator.device)
            
            with torch.no_grad():
                # Don't pass forced_bos_token_id, use model config
                generated = accelerator.unwrap_model(model).generate(
                    **inputs,
                    max_length=config.max_length,
                    num_beams=config.num_beams,
                    early_stopping=True
                )
            
            decoded = tokenizer.batch_decode(generated, skip_special_tokens=True)
            predictions.extend(decoded)
        
        # Compute COMET
        comet_input = {
            "sources": sources,
            "predictions": predictions,
            "references": references
        }
        
        comet_score = comet_metric.compute(**comet_input, model_name="Unbabel/wmt22-comet-da")
        
        print(f"\n✓ Final COMET Score: {comet_score['mean_score']:.4f}")
        print(f"✓ Final BLEU Score: {best_bleu:.2f}")
        
    except Exception as e:
        print(f"⚠️  Could not compute COMET score: {e}")
        print("This is normal on Kaggle due to resource constraints.")
        print("COMET requires significant memory and may timeout.")



Running final evaluation with COMET...


⚠️  Could not compute COMET score: To be able to use evaluate-metric/comet, you need to install the following dependencies['comet'] using 'pip install unbabel-comet' for instance'
This is normal on Kaggle due to resource constraints.
COMET requires significant memory and may timeout.


In [48]:
if accelerator.is_main_process:
    print("\n" + "=" * 80)
    print("Saving final model...")
    
    # Ensure the best model is saved
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(config.output_dir)
    tokenizer.save_pretrained(config.output_dir)
    
    # Save config
    with open(os.path.join(config.output_dir, "training_config.json"), "w") as f:
        json.dump(vars(config), f, indent=2)
    
    print(f"✓ Final model saved to: {config.output_dir}")
    print("=" * 80)


Saving final model...


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'forced_bos_token_id': 256011}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


✓ Final model saved to: /kaggle/working/nllb-fr-ar-legal


In [49]:
if accelerator.is_main_process:
    print("\n" + "=" * 80)
    print("Testing inference with fine-tuned model...")
    print("=" * 80)
    
    # Reload model for inference
    inference_model = AutoModelForSeq2SeqLM.from_pretrained(config.output_dir)
    inference_tokenizer = AutoTokenizer.from_pretrained(config.output_dir)
    inference_model.to(accelerator.device)
    inference_model.eval()
    
    # Get forced_bos_token_id
    if hasattr(inference_tokenizer, 'lang_code_to_id'):
        tgt_forced_bos = inference_tokenizer.lang_code_to_id[config.tgt_lang]
    else:
        tgt_forced_bos = inference_tokenizer.convert_tokens_to_ids(config.tgt_lang)
    
    # Test samples
    test_texts = [
        "Article 1er : La loi régit l'ensemble du territoire national.",
        "Le code civil régit les droits patrimoniaux et extrapatrimoniaux.",
        "Conformément aux dispositions du journal officiel, le décret entre en vigueur immédiatement."
    ]
    
    print("\n🔍 Translation Examples:\n")
    
    for i, text in enumerate(test_texts, 1):
        # Tokenize
        inference_tokenizer.src_lang = config.src_lang
        inputs = inference_tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=config.max_length
        ).to(accelerator.device)
        
        # Generate
        with torch.no_grad():
            generated = inference_model.generate(
                **inputs,
                forced_bos_token_id=tgt_forced_bos,
                max_length=config.max_length,
                num_beams=5,
                early_stopping=True
            )
        
        # Decode
        translation = inference_tokenizer.decode(generated[0], skip_special_tokens=True)
        
        print(f"Example {i}:")
        print(f"  FR: {text}")
        print(f"  AR: {translation}")
        print()
    
    print("=" * 80)
    print("✅ Training and evaluation complete!")
    print(f"📁 Model location: {config.output_dir}")
    print("=" * 80)



Testing inference with fine-tuned model...

🔍 Translation Examples:

Example 1:
  FR: Article 1er : La loi régit l'ensemble du territoire national.
  AR: المادة 1: القانون ينظم كل الأراضي الوطنية.

Example 2:
  FR: Le code civil régit les droits patrimoniaux et extrapatrimoniaux.
  AR: القانون المدني ينظم الحقوق الممتلكة وغير الممتلكة.

Example 3:
  FR: Conformément aux dispositions du journal officiel, le décret entre en vigueur immédiatement.
  AR: وفقًا لأحكام الجريدة الرسمية، فإن المرسوم ينطبق على الفور.

✅ Training and evaluation complete!
📁 Model location: /kaggle/working/nllb-fr-ar-legal


In [50]:

"""
# ========================================================================
# USING THE FINE-TUNED MODEL
# ========================================================================

# Load the model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/kaggle/working/nllb-fr-ar-legal"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Translation function
def translate_french_to_arabic(text, max_length=256):
    tokenizer.src_lang = "fra_Latn"
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Get forced_bos_token_id
    if hasattr(tokenizer, 'lang_code_to_id'):
        forced_bos = tokenizer.lang_code_to_id["arb_Arab"]
    else:
        forced_bos = tokenizer.convert_tokens_to_ids("arb_Arab")
    
    generated = model.generate(
        **inputs,
        forced_bos_token_id=forced_bos,
        max_length=max_length,
        num_beams=5,
        early_stopping=True
    )
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)

# Example usage
french_text = "Article 5 : Toute personne a droit à la liberté et à la sécurité."
arabic_translation = translate_french_to_arabic(french_text)
print(f"French: {french_text}")
print(f"Arabic: {arabic_translation}")
"""

print("\n✅ Notebook execution complete!")


✅ Notebook execution complete!
